In [1]:
import numpy as np
import opt_einsum as oe
from copy import deepcopy
import itertools
from scipy import integrate
import pickle as pk
import copy
import math
import scipy

import torch
torch.set_default_dtype(torch.float64)

import os
import sys
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy'))
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/tnpy/tnpy'))

import tnpy as tp

In [ ]:
%reload_ext autoreload
%autoreload all

t = torch.randn(4, 3)
print(t)

u, s, v = torch.linalg.svd(t, full_matrices=False)
print(u.shape, s.shape, v.shape)
print(u)
print(s)
print(v)

print(torch.linalg.norm((u @ s.diag() @ v)-t))

u, s, v = tp.linalg.svd(t)
print(u.shape, s.shape, v.shape)
print(u)
print(s)
print(v)

print(torch.linalg.norm((u @ s.diag() @ v)-t))

In [95]:
# Heisenberg model
# exact GS energy: E=-0.66944

%reload_ext autoreload
%autoreload all

chi = 4
rho = 8
nx, ny = 2, 2

tps = tp.tps.SquareTPS.rand(nx=nx, ny=ny, chi=chi, rho=rho, cflag=False)
print(tps.coords, tps.nx, tps.ny)

xyz = tp.models.SquareXYZ(Jx=1.0, Jy=1.0, Jz=1.0, cflag=False)
ham_xyz = xyz.twobody_ham()
print(ham_xyz)
teo = xyz.twobody_img_time_evo(delta=0.1)

((0, 0), (1, 0), (0, 1), (1, 1)) 2 2
tensor([[[[ 0.25000,  0.00000],
          [ 0.00000, -0.25000]],

         [[ 0.00000,  0.00000],
          [ 0.50000,  0.00000]]],


        [[[ 0.00000,  0.50000],
          [ 0.00000,  0.00000]],

         [[-0.25000,  0.00000],
          [ 0.00000,  0.25000]]]])


In [96]:
deltas = [1E-2, 1E-3, 1E-4]
nums = [1000, 1000, 1000]

counter = 0
for d, n in zip(deltas, nums):

    teo = xyz.twobody_img_time_evo(delta=d)
    u, s, v = tp.linalg.tsvd(teo, group_dims=((0, 2), (1, 3)), svd_dims=(1, 0))
    ss = torch.sqrt(s).diag()
    us = torch.einsum('abc,bB->aBc', u, ss)
    sv = torch.einsum('Aa,abc->Abc', ss, v)
    te_mpo = us, sv

    # print(te_mpo)

    for l in range(n):
        old_lts = tps.link_tensors()
        tps.simple_update_proj(te_mpo)
        counter += 1

        if l % 500 == 0:
            new_lts = tps.link_tensors()
            diff = 0.0
            for key, value in new_lts.items():
                diff += torch.linalg.norm(value[0]-old_lts[key][0])
                diff += torch.linalg.norm(value[1]-old_lts[key][1])
            print(d, l, diff)

            print(tps.beta_twobody_measure(ham_xyz).item())

0.01 0 tensor(8.70003)
tensor([0.24798, 0.24959, 0.24507, 0.24949, 0.24559, 0.24788, 0.24702, 0.24946])
0.2477595278782911
0.01 500 tensor(0.00136)
tensor([-0.26232, -0.26254, -0.26304, -0.26278, -0.26277, -0.26350, -0.26347,
        -0.26373])
-0.2630165894135766
0.001 0 tensor(0.00892)
tensor([-0.26302, -0.26322, -0.26381, -0.26336, -0.26329, -0.26428, -0.26410,
        -0.26443])
-0.26368755049906223
0.001 500 tensor(3.70003e-06)
tensor([-0.26356, -0.26360, -0.26370, -0.26363, -0.26363, -0.26380, -0.26377,
        -0.26383])
-0.26368934393106164
0.0001 0 tensor(0.00089)
tensor([-0.26364, -0.26366, -0.26373, -0.26368, -0.26367, -0.26378, -0.26376,
        -0.26380])
-0.2637169716374284
0.0001 500 tensor(2.81223e-07)
tensor([-0.26366, -0.26367, -0.26373, -0.26369, -0.26368, -0.26377, -0.26375,
        -0.26378])
-0.2637169058640466


In [97]:
# TPS tensors
%reload_ext autoreload
%autoreload all

tt = tps.unified_tensor(requires_grad=True)
print(tt.shape, tt.requires_grad)

enes = tps.ctm_twobody_measure_test(op=ham_xyz)
print(enes, torch.mean(enes))

torch.Size([4, 4, 4, 4, 4, 2]) True


TypeError: list indices must be integers or slices, not str

In [22]:
%reload_ext autoreload
%autoreload all

# CTM tensors
rho = 8

# corners
cs = [torch.rand(rho, rho).requires_grad_(False) for i in range(4)]

# edges
up_es = [torch.rand(rho, rho, chi, chi) for i in range(nx)]
# down_es = [torch.rand(rho, rho, chi, chi) for i in range(nx)]
down_es = [t.clone() for t in up_es]
left_es = [torch.rand(rho, rho, chi, chi) for j in range(ny)]
# right_es = [torch.rand(rho, rho, chi, chi) for j in range(ny)]

right_es = [t.clone() for t in left_es]
# each environment tensors as a dict
temp = {'c': cs, 'u': up_es, 'd': down_es, 'l': left_es, 'r': right_es}
ctms = {}
for c in tps.coords:
    ctms.update({c: deepcopy(temp)})

tps.update_ctms(ctms)
print(tps._rho)

8


In [ ]:
# test boundary singular values
%reload_ext autoreload
%autoreload all

dts = tps.double_tensors()

old_svs = []
old_tens = []
for c in tps.coords:
    su, new_c = tps.ctmrg_move_up(ucc=c, dts=dts)
    # print(len(su), su[0])
    # sd = tps.ctmrg_move_down(ucc=c, dts=dts)
    # sr = tps.ctmrg_move_right(ucc=c, dts=dts)
    # sl = tps.ctmrg_move_left(ucc=c, dts=dts)
    old_svs.append(su)
    # old_tens.append(tps._ctms[(0, 0)])

for j in range(16):
    new_svs = []
    for c in tps.coords:
        su, cr = tps.ctmrg_move_up(ucc=c, dts=dts)
        sr, cd = tps.ctmrg_move_right(ucc=c, dts=dts)
        sd, cl = tps.ctmrg_move_down(ucc=c, dts=dts)
        sl, cu = tps.ctmrg_move_left(ucc=c, dts=dts)
        new_svs.append(su)
        # rint(c, cr, cd, cl, cu)
    
    enes = tps.ctm_twobody_measure_test(op=ham_xyz)

    # print(j)
    for i in range(tps.size):
        print(
            i, 'u:',
            torch.linalg.norm(new_svs[i][0]-old_svs[i][0]),
            torch.linalg.norm(new_svs[i][1]-old_svs[i][1]),
            torch.linalg.norm(new_svs[i][2]-old_svs[i][2]),
            )
    old_svs = new_svs
    

In [103]:
%reload_ext autoreload
%autoreload all

rho = 8

# corners
cs = [torch.rand(rho, rho) for i in range(4)]
# edges
es = [torch.rand(rho, rho, chi, chi) for i in range(4)]
temp = cs + es
ctms = {}
for i, j in itertools.product(range(nx), range(ny)):
    ctms.update({(i, j): deepcopy(temp)})

tps.update_ctms(ctms)
print(tps._rho)
enes = tps.ctm_twobody_measure(op=ham_xyz)
print(enes, torch.mean(enes))

print(tps.beta_twobody_measure(ham_xyz))

8
tensor([ 0.01262,  0.08826,  0.00362,  0.05673,  0.01221,  0.06285, -0.01493,
         0.05533]) tensor(0.03459)
tensor([-0.26367, -0.26368, -0.26373, -0.26369, -0.26369, -0.26376, -0.26375,
        -0.26377])
tensor(-0.26372)


In [105]:
# test boundary singular values
%reload_ext autoreload
%autoreload all

dts = tps.double_tensors()

# old_svals = tps.ctm_singular_vals()

for l in range(16):

    print(l)
    tps.ctmrg_move_down(dts=dts)
    tps.ctmrg_move_left(dts=dts)
    tps.ctmrg_move_up(dts=dts)
    tps.ctmrg_move_right(dts=dts)
    enes = tps.ctm_twobody_measure(op=ham_xyz)
    # print(torch.linalg.norm(tps.unified_tensor()))
    print(enes, torch.mean(enes))

    # new_svals = tps.ctm_singular_vals()

    '''
    for c in tps.coords:
        print(
            c, 'u:',
            torch.linalg.norm(new_svals[c]['u'][0]-old_svals[c]['u'][0]),
            torch.linalg.norm(new_svals[c]['u'][1]-old_svals[c]['u'][1]),
            )
    for c in tps.coords:
        print(
            c, 'd:',
            torch.linalg.norm(new_svals[c]['d'][0]-old_svals[c]['d'][0]),
            torch.linalg.norm(new_svals[c]['d'][1]-old_svals[c]['d'][1]),
            )
    '''

    # old_svals = new_svals


0
d loss: tensor(1.27995e-14)
d loss: tensor(6.73343e-10)
l loss: tensor(3.63260e-12)
l loss: tensor(2.75927e-09)
u loss: tensor(3.37653e-12)
u loss: tensor(1.67405e-07)
r loss: tensor(3.78468e-10)
r loss: tensor(1.55005e-10)
tensor([-0.21713, -2.47249, -0.21413, -0.27361, -0.27760, -0.16097, -4.55481,
        -0.43068]) tensor(-1.07518)
1
d loss: tensor(1.16779e-10)
d loss: tensor(4.71832e-10)
l loss: tensor(2.96716e-19)
l loss: tensor(1.32314e-10)
u loss: tensor(5.97331e-11)
u loss: tensor(1.55198e-07)
r loss: tensor(2.63579e-16)
r loss: tensor(2.64041e-10)
tensor([-0.22726, -0.27634, -0.24362, -0.26318,  0.65689,  0.76539, -0.22642,
         0.96003]) tensor(0.14319)
2
d loss: tensor(1.26039e-20)
d loss: tensor(3.80236e-12)
l loss: tensor(6.99427e-16)
l loss: tensor(3.04508e-10)
u loss: tensor(1.65394e-11)
u loss: tensor(1.61038e-06)
r loss: tensor(7.28199e-19)
r loss: tensor(8.78799e-12)
tensor([-0.25838, -1.83017,  0.13374,  0.39246, -0.26680, -0.18890, -0.26437,
        -0.29925]

In [ ]:
tps._ctmrg_22_left_up_rotate(dts=dts, init_ctms=ctms)
tps._ctmrg_22_right_down_rotate(dts=dts, init_ctms=ctms)

In [ ]:
ctms, old_ss_u, old_ss_d = tps.ctmrg_ud(dts=dts, init_ctms=ctms)

p = True

for j in range(64):
    for i in range(0):
        ctms = tps.ctmrg_ud(dts=dts, init_ctms=ctms)
        # enes = tps.ctm_twobody_measure(tps=tps_t, ctms=ctms, op=ham_xyz)
        # print(enes, torch.mean(enes))

    for i in range(0):
        ctms = tps.ctmrg_ud(dts=dts, init_ctms=ctms)

    ctms, new_ss_u, new_ss_d = tps.ctmrg_ud(dts=dts, init_ctms=ctms)

    if True == p:
        for i in range(2):
            print(
                i, 'u:',
                torch.linalg.norm(new_ss_u[i][0]-old_ss_u[i][0]),
                torch.linalg.norm(new_ss_u[i][1]-old_ss_u[i][1]),
                torch.linalg.norm(new_ss_u[i][2]-old_ss_u[i][2]),
                )

        for i in range(2):
            print(
                i, 'd:',
                torch.linalg.norm(new_ss_d[i][0]-old_ss_d[i][0]),
                torch.linalg.norm(new_ss_d[i][1]-old_ss_d[i][1]),
                torch.linalg.norm(new_ss_d[i][2]-old_ss_d[i][2]),
                )
        
    old_ss_u = new_ss_u
    old_ss_d = new_ss_d
        
    # enes = tps.ctm_twobody_measure(tps=tps_t, ctms=ctms, op=ham_xyz)
    # print(j, enes, torch.mean(enes))

In [ ]:
for j in range(10):
    print(j)
    for i in range(2):
        ctms_ud = tps.ctmrg_ud(dts=dts, init_ctms=ctms)
    for i in range(2):
        ctms_lr = tps.ctmrg_lr(dts=dts, init_ctms=ctms)

    cs_ud, cs_lr = ctms_ud[0], ctms_lr[0]
    cs = [0.5*(t+tt) for t, tt in zip(cs_ud, cs_lr)]
    up_es = ctms_ud[1]
    down_es = ctms_ud[2]
    left_es = ctms_lr[3]
    right_es = ctms_lr[4]

    ctms = [cs, up_es, down_es, left_es, right_es]

    enes = tps.ctm_twobody_measure(tps=tps_t, ctms=ctms, op=ham_xyz)
    print(enes, torch.mean(enes))

In [ ]:
class model_xyz(torch.nn.Module):

    # def __init__(self, tps_tensor, dtype=torch.float64, device='cpu', use_checkpoint=False):
    def __init__(self, tps_tensor: torch.tensor):
        super(model_xyz, self).__init__()

        self._tps = torch.nn.Parameter(tps_tensor)

    def forward(self, ctms: list, op: torch.tensor):

        # build double tensors
        dts = {}
        for i, c in enumerate(tps.coords):
            dts.update({c: torch.einsum('ABCDe,abcde->AaBbCcDd', self._tps[i].conj(), self._tps[i])})
            
        # a fixed number of RG to update CTM tensors
        num_rg = 0
        for i in range(num_rg):
            ctms = tps.ctmrg(dts=dts, init_ctms=ctms)

        # after RG, calculate ene
        bond_enes = tps.ctm_twobody_measure(tps=self._tps, ctms=ctms, op=op)

        return torch.mean(bond_enes)

In [ ]:
for n in range(30):

    # CTMRG to get converged

    dts = {}
    for i, c in enumerate(tps.coords):
        dts.update({c: torch.einsum('ABCDe,abcde->AaBbCcDd', tps_t[i].conj(), tps_t[i])})

    for i in range(8):
        ctms = tps.ctmrg(dts=dts, init_ctms=ctms)
        ene = tps.ctm_twobody_measure(tps=tps_t, ctms=ctms, op=ham_xyz)

    tps_t = tps_t.detach().requires_grad_(True)
    model = model_xyz(tps_t)
    old_ene = model.forward(ctms, ham_xyz)
    print(n, old_ene.item())

    # opt = torch.optim.LBFGS(model.parameters(), lr=1E-4, max_iter=10)
    opt = torch.optim.SGD(model.parameters(), lr=1E-3)

    def closure():
        opt.zero_grad()
        loss = model.forward(ctms, ham_xyz)
        loss.backward(retain_graph=True)
        loss = loss.detach()

        return loss

    loss = model.forward(ctms, ham_xyz)
    opt.zero_grad()
    loss.backward(retain_graph=True)
    opt.step()
    # loss = loss.detach()

    params = list(model.parameters())
    # print(len(params), params[0].shape)
    tps_t = params[0].clone().detach()

    # new energy
    ene = tps.ctm_twobody_measure(tps=tps_t, ctms=ctms, op=ham_xyz)
    print(n, old_ene, ene)

In [ ]:
num_opt = 10
num_rg = 5

for j in range(num_opt):
    print('opt:', j)
    old_ene = tps.ctm_twobody_measure(tps_tensor=tps_t, ctm_tensors=ctms, op=ham_xyz)

    # perform fixed number of RGs
    for i in range(num_rg):
        ctms = tps.ctmrg(tps_tensor=tps_t, init_ctms=ctms)

    # compute energy backpropagation
    ene = tps.ctm_twobody_measure(tps_tensor=tps_t, ctm_tensors=ctms, op=ham_xyz)
    ene.backward(inputs=[tps_t])

    # optimize
    optimizer = torch.optim.SGD()

    # restart a new computation graph
    tps_t = tps_t.detach().requires_grad_(True)
    ctms = ctms.requires_grad_(True)

In [ ]:
def test(tps_tensors, op, ctm_tensors):

    cs, up_es, down_es, left_es, right_es = ctm_tensors

    # SVD to MPO
    u, s, v = tp.linalg.tsvd(op, group_dims=((0, 2), (1, 3)), svd_dims=(0, 0))
    ss = torch.sqrt(s).diag()
    us = torch.einsum('Aa,abc->Abc', ss, u)
    sv = torch.einsum('Aa,abc->Abc', ss, v)

    mpo = us, sv

    mts, mts_conj = {}, {}
    for i, c in enumerate(tps.coords):
        # temp = self.merged_tensor(c)
        mts.update({c: tps_tensors[i]})
        mts_conj.update({c: tps_tensors[i].conj()})

    res = 0.0
    for i, c in enumerate(tps.coords):
        res += torch.einsum('abcde,abcde', mts[c], mts_conj[c])

    return torch.einsum('abcde,abcde', mts[(0, 0)], mts_conj[(0, 0)])

    '''
    pair = (0, 0), (1, 0)

    # mps_u = [t.clone().detach() for t in up_es]
    mps_u = [t for t in up_es]
    mps_u.insert(0, cs[2])
    mps_u.append(cs[3])

    # mps_d = [t.clone().detach() for t in down_es]
    mps_d = [t for t in down_es]
    mps_d.insert(0, cs[0])
    mps_d.append(cs[1])

    # build pure and impure double tensors
    pure_dts = [
            torch.einsum('ABCDe,abcde->AaBbCcDd', mts_conj[pair[0]], mts[pair[0]]),
            torch.einsum('ABCDe,abcde->AaBbCcDd', mts_conj[pair[1]], mts[pair[1]])]

    impure_dts = [
            torch.einsum('ABCDE,fEe,abcde->AaBbCfcDd', mts_conj[pair[0]], mpo[0], mts[pair[0]]),
            torch.einsum('ABCDE,fEe,abcde->AfaBbCcDd', mts_conj[pair[1]], mpo[1], mts[pair[1]])]

    temp_num = torch.einsum('ab,bcde,fc->afde', mps_d[0], left_es[0], mps_u[0])
    
    temp_num = torch.einsum('egAa,efDd,AaBbCicDd,ghBb->fhCic', temp_num, mps_d[1], impure_dts[0], mps_u[1])
    temp_num = torch.einsum('egAia,efDd,AiaBbCcDd,ghBb->fhCc', temp_num, mps_d[2], impure_dts[1], mps_u[2])

    num = torch.einsum('fhCc,fi,ijCc,hj', temp_num, mps_d[3], right_es[0], mps_u[3])

    pair = (0, 0), (1, 0)

    mps_u = [t.clone().detach() for t in up_es]
    # mps_u = up_es
    mps_u.insert(0, cs[2])
    mps_u.append(cs[3])

    mps_d = [t.clone().detach() for t in down_es]
    # mps_d = down_es
    mps_d.insert(0, cs[0])
    mps_d.append(cs[1])

    # build pure and impure double tensors
    pure_dts = [
            torch.einsum('ABCDe,abcde->AaBbCcDd', mts_conj[pair[0]], mts[pair[0]]),
            torch.einsum('ABCDe,abcde->AaBbCcDd', mts_conj[pair[1]], mts[pair[1]])]

    impure_dts = [
            torch.einsum('ABCDE,fEe,abcde->AaBbCfcDd', mts_conj[pair[0]], mpo[0], mts[pair[0]]),
            torch.einsum('ABCDE,fEe,abcde->AfaBbCcDd', mts_conj[pair[1]], mpo[1], mts[pair[1]])]

    # denominator
    temp_den = torch.einsum('ab,bcde,fc->afde', mps_d[0], left_es[0], mps_u[0])
    # temp_num = temp_den.clone().detach()
    # temp_num = torch.einsum('ab,bcde,fc->afde', mps_d[0], left_es[0], mps_u[0])
    
    temp_den = torch.einsum('egAa,efDd,AaBbCcDd,ghBb->fhCc', temp_den, mps_d[1], pure_dts[0], mps_u[1])
    temp_den = torch.einsum('egAa,efDd,AaBbCcDd,ghBb->fhCc', temp_den, mps_d[2], pure_dts[1], mps_u[2])

    den = torch.einsum('fhCc,fi,ijCc,hj', temp_den, mps_d[3], right_es[0], mps_u[3])

    # numerator
    # temp_num = torch.einsum('egAa,efDd,AaBbCicDd,ghBb->fhCic', temp_num, mps_d[1], impure_dts[0], mps_u[1])
    # temp_num = torch.einsum('egAia,efDd,AiaBbCcDd,ghBb->fhCc', temp_num, mps_d[2], impure_dts[1], mps_u[2])

    # num = torch.einsum('fhCc,fi,ijCc,hj', temp_num, mps_d[3], right_es[0], mps_u[3])

    # print(num.item(), den.item(), num / den)
    '''

    return num

def test_2(tps_tensors):

    mts, mts_conj = {}, {}
    for i, c in enumerate(tps.coords):
        # temp = self.merged_tensor(c)
        mts.update({c: tps_tensors[i]})
        mts_conj.update({c: tps_tensors[i].conj()})
    res = 0.0
    for i, c in enumerate(tps.coords):
        res += torch.einsum('abcde,abcde', mts[c], mts_conj[c])

    # return torch.einsum('abcde,abcde', tps_tensors[0], tps_tensors[0].conj())
    return res

In [ ]:
tps_ts = []
for c in tps.coords:
    tps_ts.append(tps.merged_tensor(c).requires_grad_(True))

ene = tps.ctm_twobody_measure(tps_tensors=tps_ts, op=ham_xyz, ctm_tensors=ctm_ts)
# ene = test_2(tps_ts)
print(ene.item())

# print(tps_ts[0])

ene.backward(inputs=tps_ts)


In [ ]:
print(tps_ts[0].grad)

In [ ]:

lr = 1E-4
for i in range(10000):
    ene = tps.ctm_twobody_measure(tps_tensors=tps_ts, op=ham_xyz, ctm_tensors=ctm_ts)
    ene.backward(inputs=tps_ts)

    # update
    for i in range(2):
        tps_ts[i] = tps_ts[i] - lr
    if 0 == i % 200:
        print(i, c.item())